In [55]:
set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [41]:
import openai
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from typing import Optional
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.document_loaders import WebBaseLoader

In [4]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment:str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language:str = Field(description="language of text (should be ISO 639-1 code)")

In [5]:
convert_pydantic_to_openai_function(Tagging)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'type': 'object',
  'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [6]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Think carefully, and then tag the text as instructed"),
    ("user","{input}")
])

In [12]:
model = ChatOpenAI()

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
model_with_functions =  model.bind(functions=tagging_functions)

In [16]:
tagging_chain = prompt | model_with_functions

In [17]:
tagging_chain.invoke({"input":"I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-d574e29b-4115-4cf1-90c7-56dd2e6c60f6-0')

In [18]:
tagging_chain.invoke({"input":"I hate you bitch"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"en"}', 'name': 'Tagging'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-d5235bea-ae18-4fe0-815f-f4fe8b0ed160-0')

In [20]:
#######Extraction

In [21]:
class Person(BaseModel):
    """ Information about a Person """
    name:str = Field(description="name of a person")
    age:int =  Field(description="age of a person")
    

In [22]:
class Information(BaseModel):
    people:List[Person] = Field(description="List of info about people")

In [23]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'people': {'description': 'List of info about people',
    'type': 'array',
    'items': {'description': 'Information about a Person ',
     'type': 'object',
     'properties': {'name': {'description': 'name of a person',
       'type': 'string'},
      'age': {'description': 'age of a person', 'type': 'integer'}},
     'required': ['name', 'age']}}},
  'required': ['people']}}

In [24]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]

In [37]:
extraction_model = model.bind(functions = extraction_functions,function_call={"name": "Information"})

In [27]:
extraction_model.invoke("praveen is 29 but not sure of his mom laxmi but his wife sahasra is also 29")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Praveen","age":29},{"name":"Laxmi","age":null},{"name":"Sahasra","age":29}]}', 'name': 'Information'}}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 97, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-c368d4b4-59e6-4cfc-86eb-8f7586ec228d-0')

In [28]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [29]:
extraction_chain = prompt | extraction_model

In [38]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [40]:
extraction_chain.invoke({"input": "praveen is 29 but not sure of his mom laxmi but his wife sahasra is also 29"})

{'people': [{'name': 'praveen', 'age': 29}, {'name': 'sahasra', 'age': 29}]}

In [42]:
###################

In [43]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [52]:
page_content = documents[0].page_content

In [48]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [49]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [53]:
tagging_chain.invoke({"input": page_content})

{'summary': 'The article discusses the concept of building autonomous agents powered by LLM (large language model) as the core controller. It covers components such as planning, memory, and tool use, along with case studies and proof-of-concept examples.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, case studies, proof-of-concept'}